In [49]:
from stockfish import Stockfish
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import List
from evaluation.static_evaluation import evaluate_board

In [50]:
from stockfish import Stockfish

# Đường dẫn đến file thực thi Stockfish
stockfish_path = ".venv/bin/stockfish"  # Cần thay đường dẫn đúng trên hệ thống của bạn

stockfish = Stockfish(stockfish_path)
stockfish.set_skill_level(10)  # Đặt mức độ kỹ năng từ 0-20 (10 là trung bình)
stockfish.set_depth(15)  # Độ sâu tìm kiếm


In [51]:
test_fens = [
    "rnbqkb1r/pppppppp/5n2/8/8/5N2/PPPPPPPP/RNBQKB1R w KQkq - 0 2",  # Bình thường
    "rnbqkb1r/pppppppp/5n2/8/3P4/5N2/PPP1PPPP/RNBQKB1R b KQkq - 0 2",  # Trắng kiểm soát trung tâm
    "r1bqkbnr/pppp1ppp/8/4p3/8/8/PPPPPPPP/RNBQKBNR w KQkq e6 0 2",  # Đen đi trước và có tốt ở e6
]

In [52]:
def get_stockfish_score(fen: str) -> float:
    stockfish.set_fen_position(fen)
    score = stockfish.get_evaluation()
    return score["value"] if score["type"] == "cp" else (1000 if score["value"] > 0 else -1000)

stockfish_scores = [get_stockfish_score(fen) for fen in test_fens]
print(stockfish_scores)  # Điểm số của Stockfish cho từng thế cờ


[25, 17, 466]


In [53]:
def fitness_function(weights: List[float]) -> float:
    total_error = 0
    for fen, sf_score in zip(test_fens, stockfish_scores):
        eval_score = evaluate_board(fen, weights)
        total_error += abs(eval_score - sf_score)
    return -total_error  # Càng gần giá trị của Stockfish thì fitness càng cao


In [54]:
def initialize_population(size: int):
    return np.random.uniform(low=[0.5, 0, 0, 0, 0], 
                             high=[10, 5, 3, 2, 3], 
                             size=(size, 5))  # 5 tham số cần tối ưu

population = initialize_population(50)  # 50 cá thể
print(population)


[[6.52535717e+00 4.61530063e+00 5.76158240e-01 1.03358797e+00
  6.05923821e-01]
 [5.23068053e+00 4.10158781e+00 1.75687211e+00 1.30089522e+00
  6.96659719e-01]
 [5.54562257e+00 1.44608280e+00 1.87275501e+00 7.02275970e-01
  2.67892229e+00]
 [4.27427326e+00 1.13729447e-01 2.37054230e+00 1.42668818e+00
  1.78071322e+00]
 [2.35618788e+00 4.18525038e+00 2.44183217e+00 2.60476604e-01
  1.19273500e+00]
 [1.88209438e+00 1.73974275e+00 9.51653441e-01 4.57951737e-01
  8.07406057e-01]
 [1.77770511e+00 1.23068460e-01 1.48759071e+00 1.85769754e+00
  1.09949922e+00]
 [4.43683564e+00 2.93869029e+00 8.43328452e-02 8.80526638e-01
  2.77763630e+00]
 [3.60556398e+00 1.10480006e+00 1.92222132e+00 8.67405708e-01
  2.36922137e+00]
 [8.94655607e+00 2.14715524e+00 1.49772171e+00 1.76626048e+00
  8.53383044e-01]
 [8.18848652e+00 2.93697777e+00 9.36993644e-02 4.74958362e-01
  3.12382617e-01]
 [9.35796043e+00 3.80696378e+00 2.93211736e+00 6.43024172e-03
  1.16922075e+00]
 [4.35760266e+00 1.69672405e+00 8.180243

In [55]:
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    return np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

In [56]:
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        mutation_index = random.randint(0, len(individual)-1)
        individual[mutation_index] += np.random.uniform(-0.5, 0.5)
    return individual

In [58]:
for generation in range(100):
    fitness_scores = [fitness_function(ind) for ind in population]
    sorted_indices = np.argsort(fitness_scores)[::-1]
    population = population[sorted_indices[:20]]  

    new_population = []
    for _ in range(30):
        i1, i2 = np.random.choice(len(population), 2, replace=False)
        p1, p2 = population[i1], population[i2]
        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)

    population = np.array(new_population)
    
    best_fitness = fitness_scores[sorted_indices[0]]
    print(f"Thế hệ {generation+1}: fitness tốt nhất = {best_fitness:.3f}")

best_weights = population[0]
print("Trọng số tối ưu:", best_weights)


Thế hệ 1: fitness tốt nhất = -1.541
Thế hệ 2: fitness tốt nhất = -1.541
Thế hệ 3: fitness tốt nhất = -1.541
Thế hệ 4: fitness tốt nhất = -1.541
Thế hệ 5: fitness tốt nhất = -1.541
Thế hệ 6: fitness tốt nhất = -1.541
Thế hệ 7: fitness tốt nhất = -1.541
Thế hệ 8: fitness tốt nhất = -1.541
Thế hệ 9: fitness tốt nhất = -1.541
Thế hệ 10: fitness tốt nhất = -1.541
Thế hệ 11: fitness tốt nhất = -1.541
Thế hệ 12: fitness tốt nhất = -1.541
Thế hệ 13: fitness tốt nhất = -1.541
Thế hệ 14: fitness tốt nhất = -1.541
Thế hệ 15: fitness tốt nhất = -1.541
Thế hệ 16: fitness tốt nhất = -1.541
Thế hệ 17: fitness tốt nhất = -1.541
Thế hệ 18: fitness tốt nhất = -1.541
Thế hệ 19: fitness tốt nhất = -1.541
Thế hệ 20: fitness tốt nhất = -1.541
Thế hệ 21: fitness tốt nhất = -1.541
Thế hệ 22: fitness tốt nhất = -1.541
Thế hệ 23: fitness tốt nhất = -1.541
Thế hệ 24: fitness tốt nhất = -1.541
Thế hệ 25: fitness tốt nhất = -1.541
Thế hệ 26: fitness tốt nhất = -1.541
Thế hệ 27: fitness tốt nhất = -1.541
Thế hệ 28: